In [12]:
import mhcflurry
from mhcflurry import Class1PresentationPredictor

%matplotlib inline

import re
import os
import sys
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio import Align
import matplotlib.pyplot as plt
import pandas as pd
from joblib import load
from igraph import Graph as igraph_graph, plot, rescale

# adding project folder to lib search path
project = "/".join((os.path.split(os.getcwd())[0], os.path.split(os.getcwd())[1], 'scripts'))
if project not in sys.path:
    sys.path.append(project)
    
import loader
import work_with_files
import classificator_k_mer
import tree_building
import patients_data
import data_prep_k_mer
import quantitative_analyze
import calculating_metrics

### Looking at p1

In [22]:
# loading pat

pat = patients_data.Patient('p1')

pat_reg_data = pat.regions.region.loc[pat.regions.region.name == 'V3']

In [32]:
# getting only peptides

pat_peptides = list(pat_reg_data.translated.apply(str))

In [42]:
# cutting peptides into pieces

pat_pept_pieces = []

for pepti in pat_peptides:
    i = 9
    j = 0
    
    length = len(pepti)
    
    while i < length:
        
        # will drop pieces with stop codon
        if '*' in pepti[j: i]:
            print(pepti[j: i])
            i += 2
            j += 2
            continue
        
        pat_pept_pieces.append(pepti[j: i])
        i += 2
        j += 2

NIHNSAP*R
HNSAP*RTC
SAP*RTCRN
P*RTCRNQL
RTRTSIL*N
RTSIL*NRR
SIL*NRRHN
L*NRRHNRR
KKSILCN*W
SILCN*WNK
LCN*WNKME
N*WNKMERN
ERNFKTGS*
NFKTGS*KI
KTGS*KIKK
GS*KIKKVL
*KIKKVL**
IKKVL**DN
KVL**DNSL
L**DNSL*T
*DNSL*TTL
NSL*TTLRR
L*TTLRRRS
EKLKKVL**
LKKVL**DN
KVL**DNSL
L**DNSL*T
*DNSL*TTL
NSL*TTLRR
L*TTLRRRS


In [23]:
# settings alleles

pat_hla = ['HLA-A*02:02', 'HLA-B*08:15','HLA-C*03:06']

In [13]:
predictor = Class1PresentationPredictor.load()

Using TensorFlow backend.


In [ ]:
pept_pred = predictor.predict(peptides=pat_pept_pieces,
                  alleles=pat_hla,
                  verbose=0)

pept_pred

In [45]:
pept_pred.loc[pept_pred.affinity < 1000]

,peptide,peptide_num,sample_name,affinity,best_allele,processing_score,presentation_score
2,QLLLNGSLA,2,sample1,807.826650,HLA-A*02:02,0.000725,0.094030
5,SLAEEDIII,5,sample1,34.558743,HLA-A*02:02,0.043236,0.665340
10,SENITNNAK,10,sample1,720.872168,HLA-C*03:06,0.029800,0.112990
11,NITNNAKTI,11,sample1,164.981463,HLA-C*03:06,0.014315,0.308923
12,TNNAKTIIV,12,sample1,384.053326,HLA-C*03:06,0.513307,0.555321
...,...,...,...,...,...,...,...
5621,GQVAEKLKK,5621,sample1,552.103665,HLA-C*03:06,0.060071,0.152436
5622,VAEKLKKYF,5622,sample1,83.364993,HLA-C*03:06,0.015099,0.451231
5625,KYFNKTIVF,5625,sample1,237.619802,HLA-C*03:06,0.914758,0.888902
5629,FKPHSGGDL,5629,sample1,536.525592,HLA-C*03:06,0.123958,0.188084
